<a name="top"></a><img src="source/SpinalHDL.png" alt="SpinalHDL based on Scala" style="width:320px;" />

Before running Spinal HDL code, be sure to load SpinalHDL Libraries  
**Note** : This may be a little slow when the first time load, please wait a moment to download Lib from remote.)   

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-spinal.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

## 1. val def in Trait
**Attation** val cause Error 

In [ ]:
trait PRNBase {
  val size: Int 

  val Mask = (1 << size) - 1
  val Msb  = (1 << (size - 1))
}

object GPS extends PRNBase{
    val size = 1023
}

object BD extends PRNBase{
    val size = 2046
}
BD.Mask toHexString // return 0

In [ ]:
trait PRNBase {
  val size: Int 

  def Mask = (1 << size) - 1
  def Msb  = (1 << (size - 1))
}
object BD extends PRNBase{
    val size = 11
}
BD.Mask toHexString

## 2. Scala double definition  
```scala 
class TestDoubleDef{
  def foo(p:List[String]) = {}
  def foo(p:List[Int]) = {}
}
```
**raise Error:**
```sh
[error] double definition:
[error] method foo:(List[String])Unit and
[error] method foo:(List[Int])Unit at line 120
[error] have same type after erasure: (List)Unit 
```
Solution: https://stackoverflow.com/questions/3307427/scala-double-definition-2-methods-have-the-same-type-erasure/3544060#3544060

In [ ]:
object MyTransform{
  def apply(x: Int ): Double              = x + 0.00 
  def apply(x: List[Int] ): List[Double]  = x.map(_+0.00)
  def apply(x: List[Double] ): List[Double] = x.map(_+0.00)
}

**Use implict transform**

In [ ]:
case class IntList(list: List[Int])
case class LongList(list: List[Double])
case class DoubleList(list: List[Double])

implicit def Il(list: List[Int]) = IntList(list)
implicit def Ll(list: List[Double]) = LongList(list)
implicit def Dl(list: List[Double]) = DoubleList(list)

object FixTo{
  def apply(x: Int ): Double              = x + 0.00 
  def apply(x: IntList ): List[Double]    = x.list.map(_+0.00)
  def apply(x: LongList ): List[Double]   = x.list.map(_+0.00)
  def apply(x: DoubleList ): List[Double] = x.list.map(_+0.00)
}

In [ ]:
val a = FixTo(3)
val b = FixTo(DoubleList(List(1,2,3,4,5)))
val c = FixTo(List(1,2,3,4,5))

## 3. Timethis  
```scala 
 val beg = System.currentTimeMillis()
 ... block 
 val gap = System.currentTimeMillis() - beg
 println(s"time consume $gap ms")
```

In [ ]:
 val beg = System.currentTimeMillis()

## 4. Peformance  
```scala 
 val data = List.fill(100000)(rand.nextInt)
 val sum = data.foldLeft(0)(_+_)
 val content = data.map(_.toString).foldLeft("")(_+_)  // Attation!! bad-performance
 val content = data.mkString("\n")                     // this is recommended
 import java.io.PrintWriter
 new PrintWriter("./test.txt"){write(content);close} 
```

So it's better not to use foldLeft/foldRight for string splicing

## 5. Sequence slicing
- Way1: `map(i => source(i * step))` **×**
- Way2: `source.sliding(step, step).map(_.head).toList`  **√**
- Way3: `map(i => source(i * step))` **×**

#### Way1 `15s`

In [ ]:
val rand = new scala.util.Random(0)
val source = List.fill(16384*16)(rand.nextDouble())
val step = 8
SpinalProgress("start")
//val x = (0 until source.size/step).map(i => source(i*step))
SpinalProgress("done")

#### Way2  `0.03S` recomonded
equal
```scala 
val x = source.grouped(step).map(_.head).toList
```

In [ ]:
SpinalProgress("start")
val x = source.sliding(step, step).map(_.head).toList
SpinalProgress("done")

#### Way3 `45s`

In [ ]:
SpinalProgress("start")
val x = for(i <- 0 until source.size/step) yield source(i*downTimes)
SpinalProgress("done")